In [1]:
contig = ""

In [1]:
# import google.auth
# import gcsfs

# credentials, project_id = google.auth.default()
# print(credentials)
# print(project_id)

# gcs = gcsfs.GCSFileSystem(
#       token=credentials,
#       project=project_id, 
#       cache_timeout=0, 
#       block_size=2**18,
#     )

In [2]:
import numpy as np
import ancIBD
import allel
import malariagen_data
import pandas as pd
import itertools
from tqdm.notebook import tqdm

from numba import njit

@njit()
def convert_genotypes_to_probs(genotypes, error_rate=0.01):
    snps, samples, _ = genotypes.shape
    probs = np.zeros((snps, samples, 3), dtype=np.float64)
    
    for i in range(snps):
        for j in range(samples):
            allele1, allele2 = genotypes[i, j]
            
            if allele1 == 0 and allele2 == 0:
                probs[i, j] = [1 - error_rate, error_rate / 2, error_rate / 2]
            elif allele1 == 0 and allele2 == 1:
                probs[i, j] = [error_rate / 2, 1 - error_rate, error_rate / 2]
            elif allele1 == 1 and allele2 == 0:
                probs[i, j] = [error_rate / 2, 1 - error_rate, error_rate / 2]
            else:  # allele1 == 1 and allele2 == 1
                probs[i, j] = [error_rate / 2, error_rate / 2, 1 - error_rate]
    
    return probs

def read_ag_gmap(contig):
    if contig in {"2RL", "3RL"}:
        chrom = contig[0]
        contig_r, contig_l = f"{chrom}R", f"{chrom}L"
        df_r = read_ag_gmap(contig_r)
        df_l = read_ag_gmap(contig_l)
        max_ppos = df_r["pposition"].iloc[-1]
        max_gpos = df_r["gposition"].iloc[-1]
        df_l = df_l.iloc[1:]
        df_l["pposition"] += max_ppos
        df_l["gposition"] += max_gpos
        df = pd.concat([df_r, df_l], axis=0, ignore_index=True)
    else:
        df = pd.read_csv(gmap_dict[contig], sep="\t")
    return df
    
def ag_gmap(contig):
    ag3 = malariagen_data.Ag3()
        
    # read in the genetic map file
    df_gmap = read_ag_gmap(contig)

    # set up an array of per-base recombination rate values
    rr = np.zeros(len(ag3.genome_sequence(contig)), dtype="f8")

    # fill in the recombination rate values from the genetic map file
    for row, next_row in zip(
        itertools.islice(df_gmap.itertuples(), 0, len(df_gmap)-1), 
        itertools.islice(df_gmap.itertuples(), 1, None)):
        
        # N.B., the genetic map file is in units of cM / Mbp
        # we multiple by 1e-6 to convert to cM / bp
        rr[row.pposition-1:next_row.pposition] = row.rrate * 1e-6
    
    # compute mapping from physical to genetic position
    gmap = np.cumsum(rr)
        
    return gmap
    
def ag_p2g(contig, ppos):
    """Convert physical position (bp) to genetic position (M)."""
    gmap = ag_gmap(contig)
    gpos = gmap[ppos - 1]
    return gpos / 100 # morgans not centimorgans 


gmap_dict = {contig : f"https://raw.githubusercontent.com/anopheles-genomic-surveillance/selection-atlas/main/workflow/notebooks/ag_{contig}.gmap"
            for contig in ['2L', '2R', '3L', '3R', 'X']
}


def prepare_ancIBD_input(contig, sample_sets, sample_query, cohort_size=None):
    # load haplotypes
    #ds_haps = ag3.haplotypes(region=contig, sample_sets=sample_sets, sample_query=sample_query, cohort_size=cohort_size)
    import zarr

    df_crosses = ag3.cross_metadata().query("role == 'progeny'")
    samples = df_crosses['sample_id']
    
    gt = zarr.load(f"crosses-offspring-gn-{contig}.zarr")
    pos = zarr.load(f"pos-{contig}.zarr")

    # load arrays
    gt = allel.GenotypeArray(gt)   #ds_haps['call_genotype'].values)
    # pos = ds_haps['variant_position'].values
    # samples = ds_haps['sample_id'].values

    # find segregating sites
    seg_ = gt.count_alleles().is_segregating()
    gt = gt.compress(seg_, axis=0)
    pos = pos[seg_]

    # find the alternate allele frequency
    frqs = gt.count_alleles().to_frequencies()
    alt_frqs = frqs[:, 1]
    
    # convert GTs to genotype probabilities
    gp = convert_genotypes_to_probs(gt.values)

    # prepare input dictionary
    input_dict = {
        'calldata/GT':gt.values,
        'calldata/GP':gp,
        'calldata/AF': alt_frqs,
        'variants/POS':pos,
        'variants/MAP':ag_p2g(contig=contig, ppos=pos),
        'samples':samples
    }
    
    return input_dict

# vectorized haversine function
def haversine(lat1, lon1, lat2, lon2, to_radians=True, earth_radius=6371):
    """
    slightly modified version: of http://stackoverflow.com/a/29546836/2901002

    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees or in radians)

    All (lat, lon) coordinates must have numeric dtypes and be of equal length.

    """
    if to_radians:
        lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])

    a = np.sin((lat2-lat1)/2.0)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2

    return earth_radius * 2 * np.arcsin(np.sqrt(a))

/home/snagi/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### I need to prepare a dict containing multiple arrays for a given contig:

- calldata/GT
- calldata/GP
- genomic positions
- linkage map
- samples

Lets try and create these directly from malariagen_data. 

In [3]:
ag3 = malariagen_data.Ag3(results_cache="../results_cache/")

sample_sets = "1244-VO-GH-YAWSON-VMF00149"
sample_query = None

In [4]:
contig = '2RL'

input_dict = prepare_ancIBD_input(contig=contig, sample_sets=sample_sets, sample_query=sample_query)

In [ ]:
from ancIBD.run import hapBLOCK_chroms

df_ibd = hapBLOCK_chroms(folder_in=input_dict,
                         iids=input_dict['samples'], 
                         run_iids=[],
                         ch=contig, 
                         folder_out="ancIBD-cross",
                         output=False, 
                         prefix_out='', 
                         logfile=False,
                         l_model='dict',
                         e_model='haploid_gl2', 
                         h_model='FiveStateScaled', 
                         t_model='standard',
                         p_col='calldata/AF',
                         ibd_in=1,
                         ibd_out=10, 
                         ibd_jump=400,
                         min_cm=1,
                         cutoff_post=0.99,
                         max_gap=0.0075)

df_ibd = df_ibd.assign(size=lambda x: x.EndBP - x.StartBP)

0it [00:00, ?it/s]

In [ ]:
df_samples = ag3.sample_metadata(sample_sets=sample_sets, sample_query=sample_query)

In [ ]:
df_loc = df_samples[['sample_id', 'location', 'latitude', 'longitude']]#''.groupby('location').agg({'latitude':'mean', 'longitude':'mean'}).reset_index()
df_loc = df_loc.assign(location=df_loc.location.str.split(".").str.get(0))

In [ ]:
# df_ibd.to_csv("df_ibd.csv")
df_ibd = pd.read_csv("../../results/ancIBD/chX.tsv", sep="\t", index_col=0)
df_ibd.shape

In [ ]:
df_ibd = df_ibd.assign(size=lambda x: x.EndBP - x.StartBP,
                       cm=lambda x: x.lengthM * 100,
                       ).sort_values("size", ascending=False)

In [ ]:
df_ibd = df_ibd.query("cm > 12")
print(df_ibd.shape)


n_segments = df_ibd.groupby(['iid1', 'iid2']).size()

df_ibd_agg = df_ibd.groupby(['iid1', 'iid2']).agg({'cm':'sum'}).reset_index()
df_ibd_agg.shape



In [ ]:
df_samples = df_samples.assign(location=df_samples.location.str.split(".").str.get(0))

# check are iid1 and iid2 always the same location in df_samples
loc1s = []
loc2s = []
dists = []
for i, row in tqdm(df_ibd_agg.iterrows()):
    loc1 = df_samples[df_samples.sample_id == row.iid1].location.values[0]
    loc2 = df_samples[df_samples.sample_id == row.iid2].location.values[0]
    loc1s.append(loc1)
    loc2s.append(loc2)

    # calc geographic distance
    lat1, lon1 = df_samples[df_samples.sample_id == row.iid1][['latitude', 'longitude']].values[0]
    lat2, lon2 = df_samples[df_samples.sample_id == row.iid2][['latitude', 'longitude']].values[0]
    dist = haversine(lat1, lon1, lat2, lon2)
    dists.append(dist)

df_ibd_agg = df_ibd_agg.assign(loc1=loc1s, loc2=loc2s)

In [ ]:
import plotly.express as px

px.scatter_3d(
    df_ibd_agg.assign(geographic_distance=dists), 
    x='geographic_distance', 
    y='cm',
    z=n_segments, 
    template='simple_white', 
    hover_data=['iid1', 'iid2']
    )

In [ ]:
# from bokeh.io import output_notebook # enables plot interface in J notebook
# output_notebook(hide_banner=True)

In [ ]:
# def plot_ibd_segments_contig(
#     self,
#     df,
#     contig, 
#     filter_expr=pl.col('n_snps') > 10_000,
#     show=False,
#     width=None,
#     height=400
# ):
#     from itertools import combinations
#     import bokeh.plotting as bkplt
#     import bokeh.models as bkmod
#     import bokeh.layouts as bklay
    
#     df = df.filter(pl.col('contig') == contig).filter(filter_expr)
        
#     print("finding levels")
#     max_inds = df['idx1'].max() + 1
#     level_df = pl.DataFrame(list(combinations(range(max_inds), 2))).transpose()
#     level_df = level_df.with_columns(pl.lit(np.linspace(0, 1, len(list(combinations(range(max_inds), 2))))).alias("level"))
#     level_df = level_df.rename({'column_0':'idx1', 'column_1':'idx2'}).with_columns(pl.col("idx1").cast(pl.Int32),
#                                                                                    pl.col("idx2").cast(pl.Int32))
#     df = df.join(level_df, on=['idx1', 'idx2'])
    
# #     print("colour mapping")
# #     colour_mapping = {'half-ibd':'gray', 
# #                       'full-ibd':'blue'}
# #     colour = df['ibd_type'].apply(lambda x: colour_mapping[x])

#     source = bkmod.ColumnDataSource(data={
#         'index1': df['idx1'].to_numpy(),
#         'index2': df['idx2'].to_numpy(),
#          'sample_id1':df['sample_id1'].to_numpy(),
#          'sample_id2':df['sample_id2'].to_numpy(),
#         'chromosome': df['contig'].to_numpy(),
#         'start': df['start'].to_numpy(),
#         'end': df['end'].to_numpy(),
#         'bottoms':df['level'].to_numpy(),
#         'tops': df['level'].to_numpy()+0.0001,
# #         'colour': colour.to_numpy()
#     })

#     hover = bkmod.HoverTool(tooltips=[
#             ("index1", '@index1'),
#             ("index2", '@index2'),
#              ("sample_id1", '@sample_id1'),
#              ("sample_id2", '@sample_id2'),
#             ("segment span", "@start{,} - @end{,}"),
#         ])
        
#     print("making figure")
#     if not width:
#         width = int(self.genome_sequence(contig).shape[0]/200000)
#     fig1 = bkplt.figure(title=contig,
#                         width=width,
#                         height=500, 
#                         tools="tap,box_zoom,xpan,xzoom_in,xzoom_out,xwheel_zoom,reset".split() + [hover],
#                         toolbar_location='above', active_drag='xpan', active_scroll='xwheel_zoom',
#                         output_backend="webgl")

#     glyph = bkmod.Quad(left='start', right='end', bottom='bottoms', top='tops', line_color="grey", line_alpha=.8, line_width=1)
#     fig1.add_glyph(source, glyph)

#     fig1.x_range = bkmod.Range1d(0, self.genome_sequence(contig).shape[0], bounds='auto')
#     fig1.y_range = bkmod.Range1d(0, 1, bounds='auto')
#     fig1.x_range.max_interval = self.genome_sequence(contig).shape[0]
#     fig1.yaxis.visible = False
#     fig1.ygrid.visible = False
#     _bokeh_style_genome_xaxis(fig1, contig)
    
#     if show:
#         bkplt.show(fig1)
    
#     return fig1

# def plot_ibd_segments(
#         self,
#         df, 
#         out_dir,
#         cohort_id,
#         contigs=('2RL', '3RL', 'X'),
#         filter_expr=pl.col('n_snps') > 10_000,
#         show=True,
#         title=None,
#     ):
#     import bokeh.models as bkmod
#     import bokeh.layouts as bklay
#     import bokeh.plotting as bkplt
    
#     figs = [
#             plot_ibd_segments_contig(
#                 self=self,
#                 df=df,
#                 contig=contig,
#                 filter_expr=filter_expr,
#                 ) 
#             for contig in tqdm(contigs)
#             ]
    
#     if out_dir:
#         bkplt.output_file(filename=out_dir + cohort_id + "_segments.html", title=title)

#     fig = bklay.gridplot(
#         figs,
#         ncols=len(contigs),
#         toolbar_location="above",
#         merge_tools=True,
#     ) 
    
#     if out_dir:
#         bkplt.save(fig)
    
#     if show:
#         bkplt.show(fig)
    
#     return fig
    
# def _bokeh_style_genome_xaxis(fig, contig):
#     import bokeh.models as bkmod
#     """Standard styling for X axis of genome plots."""
#     fig.xaxis.axis_label = f"Contig {contig} position (bp)"
#     fig.xaxis.ticker = bkmod.AdaptiveTicker(min_interval=1)
#     fig.xaxis.minor_tick_line_color = None
#     fig.xaxis[0].formatter = bkmod.NumeralTickFormatter(format="0,0")